### Generating Synthetic Data using Sudoku

In [1]:
import json

from sympy.core.random import choice

from utils.sudoku import SudokuEnvironment

game = SudokuEnvironment()
game.create_puzzle(difficulty='easy')
game.display()
deductions = game.get_explicit_deductions()
print(deductions[0])

+-------+-------+
| _ 2 | 3 _ |
| _ 4 | 1 _ |
+-------+-------+
| 2 1 | 4 3 |
| 4 3 | _ _ |
+-------+-------+
['Position (0,0) must be 1 (only valid value  because 10 - 2 - 3 - 4 = 1)', 'Position (0,3) must be 4 (only valid value  because 10 - 1 - 2 - 3 = 4)', 'Position (1,0) must be 3 (only valid value  because 10 - 1 - 2 - 4 = 3)', 'Position (1,3) must be 2 (only valid value  because 10 - 1 - 3 - 4 = 2)', 'Position (3,2) must be 2 (only valid value  because 10 - 1 - 3 - 4 = 2)', "Position (3,3) must be 1 (only position in row 3 for 1 because missing 1, 2 from unit; 1 can't go in (3,2) blocked by col 2)"]


#### Build Data generation pipeline

##### Build question

In [2]:
import random
import copy

def randomize_dict_values(dict_list, seed=None):
    """
    Replace each 'value' with a random different value from range 1-4.
    The new value is guaranteed to be different from the original.
    All other keys remain unchanged.

    Args:
        dict_list: List of dictionaries containing 'value' key
        seed: Random seed for reproducibility (optional)

    Returns:
        List of dictionaries with 'value' key randomized to different values
    """
    if not dict_list:
        return []

    if seed is not None:
        random.seed(seed)

    # Create deep copy to avoid modifying the original
    randomized_list = copy.deepcopy(dict_list)

    # Replace each value with a random different value
    for new_dict in randomized_list:
        original_value = new_dict['value']

        # Create possible values (1-4) excluding the original value
        possible_values = [v for v in range(1, 5) if v != original_value]

        # Randomly select from the remaining values
        new_dict['value'] = random.choice(possible_values)

    return randomized_list

def generate_answer_options(correct_deduction, wrong_deductions):
    """
    Generate multiple-choice answer options with one correct and several wrong answers.

    Args:
        correct_deduction: Dictionary containing the correct deduction
        wrong_deductions: List of dictionaries containing wrong deductions

    Returns:
        tuple: (formatted_answer_string, correct_answer_letter)
    """
    answer_options_list = []
    answer_options_set = {'A', 'B', 'C', 'D'}

    # Select a random option for the correct answer
    correct_answer_option = random.choice(list(answer_options_set))
    answer_options_set.discard(correct_answer_option)

    # Add correct answer to the list
    correct_text = f"{correct_answer_option}) Position ({correct_deduction['row']},{correct_deduction['col']}) must be {correct_deduction['value']}"
    answer_options_list.append(correct_text)

    # print(len(wrong_deductions))
    for idx, wrong_deduction in enumerate(wrong_deductions):
        wrong_answer_option = random.choice(list(answer_options_set))
        answer_options_set.discard(wrong_answer_option)
        # print(wrong_deduction)
        wrong_text = f"{wrong_answer_option}) Position ({wrong_deduction['row']},{wrong_deduction['col']}) must be {wrong_deduction['value']}"
        answer_options_list.append(wrong_text)
        if idx == 2:
            break

    # Sort by the option letter and join into a string
    answer_options_list.sort(key=lambda x: x[0])
    formatted_answers = "\n".join(answer_options_list)

    # Extract the correct answer letter (without the closing parenthesis)
    correct_letter = correct_answer_option[0]

    return formatted_answers, correct_letter

In [3]:
puzzle = {
    "question": None,
    "question_parsing": None,
    "answer": None,
    "id": None,
    "sel_idx": None,
    "cot": None,
    "cot_parsing": None
}

In [4]:
import random
#TODO: add positions property, value property to the sudoku function

# define overall question
question = 'You have the following Sudoku puzzle which of the following answers is correct? \n'

# Append the game to the question
question += game.display(print_game=False)

# get empty cells to generate randomly wrong answers -> shuffling
deductions_data = game.get_explicit_deductions()[-1]
# print(deductions_data)

# print(deductions_data)
# shuffle values
correct_deduction = deductions_data[0]
wrong_deductions = randomize_dict_values(deductions_data[1:])
opts, answer = generate_answer_options(correct_deduction, wrong_deductions)
question += "\n" + opts
print(question)

puzzle['question'] = question
puzzle['answer'] = answer

# Get permutations -> create a questions for each of the empty fields -> for possible answers a, b, c, d create one correct answer and 3 wrong answers

You have the following Sudoku puzzle which of the following answers is correct? 
+-------+-------+
| _ 2 | 3 _ |
| _ 4 | 1 _ |
+-------+-------+
| 2 1 | 4 3 |
| 4 3 | _ _ |
+-------+-------+
A) Position (1,3) must be 1
B) Position (0,3) must be 3
C) Position (1,0) must be 2
D) Position (0,0) must be 1


#### Create Question Parsing

In [12]:
# get positions of empty fields? Get position of non empty fields? get rows, get columns and get squares?
qp_raw = game.get_all_units_as_strings()
qp_string = 'The puzzle has the rows: ' + str(qp_raw['rows'])
qp_string += '\nThe puzzle has the columns: ' + str(qp_raw['columns'])
qp_string += '\nThe puzzle has the squares: ' + str(qp_raw['squares'])

print(qp_string)

puzzle['question_parsing'] = qp_string


The puzzle has the rows: ['_ 2 3 _', '_ 4 1 _', '2 1 4 3', '4 3 _ _']
The puzzle has the columns: ['_ _ 2 4', '2 4 1 3', '3 1 4 _', '_ _ 3 _']
The puzzle has the squares: [' _2_4', ' 3_1_', ' 2143', ' 43__']



#### Generating answer

#### Generating CoT

In [18]:
# Get full deduction
cot = game.get_explicit_deductions()[0]
cot_string = "; ".join(cot)
puzzle['cot'] = cot_string

"Position (0,0) must be 1 (only valid value  because 10 - 2 - 3 - 4 = 1); Position (0,3) must be 4 (only valid value  because 10 - 1 - 2 - 3 = 4); Position (1,0) must be 3 (only valid value  because 10 - 1 - 2 - 4 = 3); Position (1,3) must be 2 (only valid value  because 10 - 1 - 3 - 4 = 2); Position (3,2) must be 2 (only valid value  because 10 - 1 - 3 - 4 = 2); Position (3,3) must be 1 (only position in row 3 for 1 because missing 1, 2 from unit; 1 can't go in (3,2) blocked by col 2)"

#### Generating CoT parsing

In [21]:
# Get statements from deduction
statements = game.get_explicit_deductions()[1]
# get evidence from deduction
evidence = game.get_explicit_deductions()[2]

# Add verification -> wrong verifications?
puzzle['cot_parsing'] = [{"statement": statements[idx], "evidence": evidence[idx],"Verification": "true"} for idx in range(len(statements))]


In [24]:
from random import randint

# add random ids
rand =  randint(0, 100000000)
puzzle['id'] = rand
puzzle['sel_idx'] = rand




In [25]:
puzzle

{'question': 'You have the following Sudoku puzzle which of the following answers is correct? \n+-------+-------+\n| _ 2 | 3 _ |\n| _ 4 | 1 _ |\n+-------+-------+\n| 2 1 | 4 3 |\n| 4 3 | _ _ |\n+-------+-------+\nA) Position (1,3) must be 1\nB) Position (0,3) must be 3\nC) Position (1,0) must be 2\nD) Position (0,0) must be 1',
 'question_parsing': "The puzzle has the rows: ['_ 2 3 _', '_ 4 1 _', '2 1 4 3', '4 3 _ _']\nThe puzzle has the columns: ['_ _ 2 4', '2 4 1 3', '3 1 4 _', '_ _ 3 _']\nThe puzzle has the squares: [' _2_4', ' 3_1_', ' 2143', ' 43__']",
 'answer': 'D',
 'id': 55521402,
 'sel_idx': 55521402,
 'cot': '',
 'cot_parsing': [{'statement': 'Position (0,0) must be 1',
   'evidence': 'Only valid value because 10 - 2 - 3 - 4 = 1',
   'Verification': 'true'},
  {'statement': 'Position (0,3) must be 4',
   'evidence': 'Only valid value because 10 - 1 - 2 - 3 = 4',
   'Verification': 'true'},
  {'statement': 'Position (1,0) must be 3',
   'evidence': 'Only valid value because 1